In [1]:
# import sys
# sys.path.append("/home/sire/phd/srz228573/torchmd-net/torchmdnet/models")

In [2]:
from torchmdnet.models.model import load_model
import numpy as np
import torch

In [3]:
#model = load_model("/home/sire/phd/srz228573/benchmarking_datasets/fone_output/torchmdnet/lips2/logs/epoch=209-val_loss=0.0165-test_loss=0.0897.ckpt", derivative=True)
model = load_model("/home/sire/phd/srz228573/benchmarking_datasets/fone_output/torchmdnet/rmd17/aspirin/logs/epoch=9-val_loss=0.1899-test_loss=0.0000.ckpt", derivative = True)

In [4]:
data_path = "/home/sire/phd/srz228573/torchmd-net/bench_data_sl/fone_dataset/mdsim_data/mace_data_sl/rmd17/npz_data/aspirin/val/nequip_npz.npz"

In [5]:
data = np.load(data_path)

In [6]:
data.files

['nuclear_charges',
 'coords',
 'energies',
 'forces',
 'old_indices',
 'old_energies',
 'old_forces']

In [14]:
data["energies"][0]

-406272.65480627894

In [7]:
data["coords"].shape

(20000, 21, 3)

In [8]:
sample_an = torch.tensor(data["nuclear_charges"], dtype=torch.int64)
sample_pos = torch.tensor(data["coords"][0], dtype = torch.float)
#sample_batch = torch.tensor(data["atomic_numbers"], dtype=torch.long)

In [9]:
pred_e , pred_f = model(sample_an, sample_pos)

In [10]:
from tqdm import tqdm

In [18]:
y_true = []
y_pred = []
for i in tqdm(range(int(len(data["coords"])*.1))):# 10% sample
    sample_an = torch.tensor(data["nuclear_charges"], dtype=torch.int64)
    sample_pos = torch.tensor(data["coords"][i], dtype = torch.float)
    pred_e , _ = model(sample_an, sample_pos)
    y_pred.append(pred_e.detach().numpy()[0][0])
    y_true.append(data["energies"][i])
    

100%|██████████| 2000/2000 [01:21<00:00, 24.51it/s]


In [19]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(y_true, y_pred)

0.26188403222357737

In [28]:
y_true = []
y_pred = []
for i in tqdm(range(int(len(data["coords"])*.1))):# 10% sample
    sample_an = torch.tensor(data["nuclear_charges"], dtype=torch.int64)
    sample_pos = torch.tensor(data["coords"][i], dtype = torch.float)
    _ , pred_f = model(sample_an, sample_pos)
    y_pred.append(pred_f.detach().numpy().reshape(-1).tolist())
    y_true.append(data["forces"][i].reshape(-1).tolist())
    

100%|██████████| 2000/2000 [01:31<00:00, 21.84it/s]


In [29]:
mean_absolute_error(y_true, y_pred)

0.3507228524127299